# Chains in LangChain

In LnagChain, a chain is a sending of a message to an LLM. It is specified by (at minimum) two things
1. an LLM
    - LangChain has APIs for LLMs. 
2. a string to send to the LLM.
    - In Langchain the strings are called <u>prompts</u>, and 
    - to <u>templatize</u> a prompt is to set it up to accept  variables.
    - LangChain has classes for types of prompts. 

In [5]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

"""The OpenAI API calls use an environment variable specified in the 
file .env and exposed through the following line. """
load_dotenv() 
our_llm = OpenAI(model = 'gpt-3.5-turbo-instruct') # I have to pay for use of this OpenAI access to ChatGPT. 

code_prompt = PromptTemplate(
    template = """
    Write a {language} function with a doc string that will {task}
    """,
    input_variables=["language","task"]
)

# Name out chain `code_chain` since it will return code.
code_chain = LLMChain(
    llm=our_llm,
    prompt=code_prompt,
)

# Call `code_chain` specifying the values for the input variables
# with a dictionary. 
result = code_chain(
    {
        "language":"python",
        "task":"return a list of positive integers less than 10"
    }
)

for k,v in result.items():
    print(k)
    print(f"  {v}\n")

language
  python
task
  return a list of positive integers less than 10
text
  
def less_than_ten():
    """Returns a list of positive integers less than 10"""
    return [1, 2, 3, 4, 5, 6, 7, 8, 9]


Running a chain returns a dictionary with 
- a key for each input variable
    - `'language'`, `'task'`
- a key for each output variable 
    - `'text'`

In [5]:
for k,v in result.items():
    print(f"Key:{k}")
    print(f"  value: {v}\n")

Key:language
  value: python

Key:task
  value: return a list of positive integers less than 10

Key:text
  value: 
def get_positive_integers_less_than_10():
    '''
    Returns a list of positive integers less than 10
    '''
    return [1, 2, 3, 4, 5, 6, 7, 8, 9]



### `output_key`
The default output key  `"text"` can be modified.

In [6]:
code_chain = LLMChain(
    llm=our_llm,
    prompt=code_prompt,
    # Change the name of the output key.
    output_key="code",
)

result = code_chain(
    {
        "language":"python",
        "task":"return a list of positive integers less than 10"
    }
)


print(f"Instead of 'text' the key for the LLM's output is \'{list(result.keys())[-1]}\'")

Instead of 'text' the key for the LLM's output is 'code'


## Sequential chains

Two or more chains can be concatenated (my words). 

In [8]:
from langchain.chains import SequentialChain # for concatenating chains

code_prompt = PromptTemplate(
    template="Write a very short {language} function that will {task}.",
    input_variables=["language","task"]
)

code_chain = LLMChain(
    llm = our_llm,
    prompt=code_prompt,
    # The ouput will be sent to the next chain, so this key matters. 
    output_key="code"
    )

test_prompt = PromptTemplate(
    input_variables=["language","code"], 
    # pass the output of the previous chain to this chain.
    template="Write a test for the following {language} code:\n{code}."
)

test_chain = LLMChain(
    llm=our_llm,
    prompt=test_prompt,
    output_key="test" # So named because the output is a python test.
)

chain = SequentialChain(
    # List the chains to put in squence in order.
    chains=[code_chain, test_chain],
    # List the variables that go into the first chain.
    input_variables=["task","language"],
    # List the variables that you want to be included in the resulting dictionary; 
    # options include 
    # (i) any input key to any chain in chains 
    # (ii) any ouput_key of any chain in chains.
    output_variables=[
        "task",
        "language",
        "code",
        'test',
        ],
)

result = chain({
    "language": "python",
    "task": "'Create a list of integers less than 10.'",
    })

# print(args)
for k,v in result.items():
    print(f'>>>> {k} <<<<')
    print(v)

>>>> language <<<<
python
>>>> task <<<<
'Create a list of integers less than 10.'
>>>> code <<<<


def create_list():
    list = [0,1,2,3,4,5,6,7,8,9]
    return list
>>>> test <<<<


1. Import the necessary testing library and the create_list function from the python code.
2. Define a function called "test_create_list" to perform the testing.
3. Within the function, use the "assert" statement to check if the returned list from the create_list function is equal to the expected list [0,1,2,3,4,5,6,7,8,9].
4. Call the test_create_list function to run the test.
5. If the assert statement returns True, the test is passed and a success message is printed. If the assert statement returns False, the test is failed and an error message is printed.
6. Include multiple assert statements to check for different expected lists, such as an empty list or a list with negative numbers.
7. Include a try-except block to catch any potential errors that may occur during the test.
8. Print a final message 